# Transitions of a $d^4$ system

We calculate the possible transitions of the type $d^4d^4\rightarrow d^3d^5$, applying some knowledge regarding the ground state of the CRO system.

To recapitulate, the representation is as follows: each level/orbital is composed of two elements of an array, one for each spin (up, down). In a low spin d4 system, we have only the t2g orbitals (xy yz xz) active leading to a 6 elements representation for a site. Two neighboring states involved in a transition are contiguous in a list, thus consisting of 12 elements.

First we generate all possible states for a single site with 3, 4, and 5 electrons.

In [1]:
import numpy as np
import itertools
import functools
import operator

In [2]:
def generate_states(electrons, states):
    seed = [1 if position < electrons else 0 for position in range(states)]
    generated_states = list(set(itertools.permutations(seed)))
    generated_states.sort(reverse=True)
    return generated_states

In [3]:
states_d3 = generate_states(3,6)
states_d4 = generate_states(4,6)
states_d5 = generate_states(5,6)
states_d4

[(1, 1, 1, 1, 0, 0),
 (1, 1, 1, 0, 1, 0),
 (1, 1, 1, 0, 0, 1),
 (1, 1, 0, 1, 1, 0),
 (1, 1, 0, 1, 0, 1),
 (1, 1, 0, 0, 1, 1),
 (1, 0, 1, 1, 1, 0),
 (1, 0, 1, 1, 0, 1),
 (1, 0, 1, 0, 1, 1),
 (1, 0, 0, 1, 1, 1),
 (0, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 0, 1),
 (0, 1, 1, 0, 1, 1),
 (0, 1, 0, 1, 1, 1),
 (0, 0, 1, 1, 1, 1)]

Not all generated states are relevant for the problem, and we can reduce the amount of states beforehand. 

From all the $d^4$ states, we consider only those with a full $d_{xy}$ orbital and those which distribute the other two electrons in different orbitals as possible initial states as those are the possible ground states. In our representations, this means only the states that have a 1 in the first two entires. Also, we do not need those states with double occupancy in zx or yz orbital.

In [28]:
possible_states_d4 = [state for state in states_d4 
                      if state[0]==1 and state[1]==1  #  dxy orbital double occupancy
                      and state[2] is not state[3]]       # dzx/dyz orbital single occupancy
possible_states_d4

[(1, 1, 1, 0, 1, 0),
 (1, 1, 1, 0, 0, 1),
 (1, 1, 0, 1, 1, 0),
 (1, 1, 0, 1, 0, 1)]

Regarding the $d^3$ states, we want at least those which keep one electron in the $d_{xy}$ orbital since we know that the others must be zero.

In [30]:
possible_states_d3 = [state for state in states_d3 if state[0]==1 or state[1]==1]
possible_states_d3

[(1, 1, 1, 0, 0, 0),
 (1, 1, 0, 1, 0, 0),
 (1, 1, 0, 0, 1, 0),
 (1, 1, 0, 0, 0, 1),
 (1, 0, 1, 1, 0, 0),
 (1, 0, 1, 0, 1, 0),
 (1, 0, 1, 0, 0, 1),
 (1, 0, 0, 1, 1, 0),
 (1, 0, 0, 1, 0, 1),
 (1, 0, 0, 0, 1, 1),
 (0, 1, 1, 1, 0, 0),
 (0, 1, 1, 0, 1, 0),
 (0, 1, 1, 0, 0, 1),
 (0, 1, 0, 1, 1, 0),
 (0, 1, 0, 1, 0, 1),
 (0, 1, 0, 0, 1, 1)]

In the case of the $d^5$ states, since our ground state has a doule occupied $d_{xy}$ orbital then it has to stay occupied.

In [31]:
possible_states_d5 = [state for state in states_d5 if state[0]==1 and state[1]==1]
possible_states_d5 

[(1, 1, 1, 1, 1, 0),
 (1, 1, 1, 1, 0, 1),
 (1, 1, 1, 0, 1, 1),
 (1, 1, 0, 1, 1, 1)]

Now we generate all possible initial $d^4d^4$

In [32]:
initial_states = list(set([tuple(functools.reduce(operator.add, combination)) 
                           for combination 
                           in itertools.combinations(possible_states_d4,2)]))
initial_states

[(1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1),
 (1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0),
 (1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1),
 (1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1),
 (1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0),
 (1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1)]

and the $d^3d^5$ final states printing out the amount of them.

In [33]:
final_states = list(set([tuple(functools.reduce(operator.add, combination)) 
                          for combination 
                          in itertools.product(possible_states_d3,possible_states_d5)]))
final_states.sort(reverse=True)
len(final_states)

64

Now we check if the transition from each initial state to each final state and print out those that are allowed.

In [34]:
def is_allowed(jump):
    if np.unique(np.remainder(np.nonzero(jump),2)).size == 1:
        return 1
    return 0

for initial_state in initial_states:
    for final_state in final_states:
        jump = np.logical_xor(initial_state,final_state)
        allowed = is_allowed(jump)
        if allowed:
            print("From initial state {} to final state {} is allowed.".format(initial_state, final_state))

From initial state (1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1) to final state (1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1) is allowed.
From initial state (1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1) to final state (1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1) is allowed.
From initial state (1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1) to final state (0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1) is allowed.
From initial state (1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1) to final state (0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1) is allowed.
From initial state (1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1) to final state (0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1) is allowed.
From initial state (1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1) to final state (0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1) is allowed.
From initial state (1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0) to final state (1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0) is allowed.
From initial state (1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0) to final state (1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0) is allowed.
From initial state (1, 1, 1, 0, 1, 0, 1,

Now we test what happens for the actual transitions. First at low temperatures we only have $S_z=\pm1$ states in the ground state in AFM ordering.



In [67]:
#                                i      '     j
#  
AFM_ground_state_multiplet    = [1,1,1,0,1,0,1,1,0,1,0,1] # d4d4 AFM state
final_state_multiplet_d3_one  = [[1,0,1,0,0,1],[1,0,0,1,1,0],[0,1,1,0,1,0]]  # 4A2 S=1/2 d3 multiplet

final_state_multiplet_one     = [tuple(functools.reduce(operator.add, [state, [1,1,1,1,0,1]])) 
                             for state 
                             in final_state_multiplet_d3_one]

for state in final_state_multiplet_one:
    jump = np.logical_xor(AFM_ground_state_multiplet, state)
    allowed = is_allowed(jump)
    print(allowed)
    

#final_state_multiplet  # d3d5 final state

0
0
1


In [69]:
AFM_ground_state_multiplet    = [1,1,0,1,0,1,1,1,1,0,1,0] # d4d4 AFM state
final_state_multiplet_d3_two  = [[1,0,0,1,0,1],[0,1,1,0,0,1],[0,1,0,1,1,0]]  # 4A2 S=1/2 d3 multiplet
final_state_multiplet_two     = [list(functools.reduce(operator.add, [state, [1,1,1,0,1,1]])) 
                             for state 
                             in final_state_multiplet_d3_two]
for state in final_state_multiplet_two:
    jump = np.logical_xor(AFM_ground_state_multiplet,state)
    allowed = is_allowed(jump)
    print(allowed)

1
0
0


From the $d^4_{\downarrow}d^4_{\uparrow}$ AFM state to the $d3d5$ $^4A_2$ $S=\frac{1}{2}$ there is only one allowed term. Comparing the ground state and the final multiplet we can see that the transition is $t_{xy,xz}/\sqrt{3}$. Unfortunatelly, this way of testing transitions is somewhat cumbersome, particularly given that we already calculated all possible combinations.

We create now all possible $^4A_2$ multiplets with $S=3/2$. These are composed by the $S_z=3/2$ and the $S_z=1/2$ states.

First for $S_z=3/2$ we have

In [50]:
multiplet_d3 = [[1,0,1,0,1,0],[0,1,0,1,0,1]] # 4A2 S=3/2 d3 multiplet, both spins
possible_states_d5 = [list(state) for state in possible_states_d5]
final_states_multiplets = [functools.reduce(operator.add, combination) 
                           for combination 
                           in itertools.product(list(multiplet_d3),list(possible_states_d5))]
final_states_multiplets.sort(reverse=True)
print("From initial state {}".format(ground_state_multiplet))
for state in final_states_multiplets:
    jump = np.logical_xor(ground_state_multiplet,state)
    allowed = is_allowed(jump)
    if allowed:
        print("       final state {} is allowed.".format(state))

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]


From this multiplet, we cannot reach any final $d^5$ state.

Doing the same for the $S_z=1/2$ states we obtain

In [51]:
multiplet_d3_down  = [[1,0,1,0,0,1],[1,0,0,1,1,0],[0,1,1,0,1,0]]  # 4A2 Sz=1/2 d3 multiplet one spin down
multiplet_d3_up  = [[1,0,0,1,0,1],[0,1,1,0,0,1],[0,1,0,1,1,0]]  # 4A2 Sz=1/2 d3 multiplet one spin up
final_states_multiplets = [functools.reduce(operator.add, combination)
                          for combination 
                          in itertools.product(multiplet_d3_down+multiplet_d3_up,possible_states_d5)]
final_states_multiplets.sort(reverse=True)
print("From initial state {}".format(ground_state_multiplet))
for state in final_states_multiplets:
    jump = np.logical_xor(ground_state_multiplet,state)
    allowed = is_allowed(jump)
    if allowed:
        print("       final state {} is allowed.".format(state))

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
       final state [1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0] is allowed.
       final state [1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1] is allowed.


Given the antiferromagnetic $d^4_\downarrow d^4_\uparrow$ state as the initial one, only transitions from the $d_{xy}$ orbital to the $d_{xz}$ or $d_{yz}$ orbitals are allowed with transition matrix element $t_{xy,xz}/\sqrt{3}$ and $t_{xy,yz}/\sqrt{3}$, respectivelly.

Now, we are interested in the $S_z=0$ $d^4$ state into account.

In [66]:
multiplet_d4_Sz0 = [[1,1,1,0,0,1],[1,1,0,1,1,0]]  # Sz=0 d4 multiplet
multiplet_d3_down  = [[1,0,1,0,0,1],[1,0,0,1,1,0],[0,1,1,0,1,0]]  # 4A2 Sz=1/2 d3 multiplet one spin down
multiplet_d3_up  = [[1,0,0,1,0,1],[0,1,1,0,0,1],[0,1,0,1,1,0]]  # 4A2 Sz=1/2 d3 multiplet one spin up
final_states_multiplets = [tuple(functools.reduce(operator.add, combination))
                          for combination 
                          in itertools.product(multiplet_d3_down+multiplet_d3_up,possible_states_d5)]
final_states_multiplets.sort(reverse=True)
for istate in multiplet_d4_Sz0:
    print("From initial state {}".format(istate))
    for state in final_states_multiplets:
        print(istate,state)
        jump = np.logical_xor(istate,state)
        allowed = is_allowed(jump)
        if allowed:
            print("       final state {} is allowed {}.".format(state, allowed))

From initial state [1, 1, 1, 0, 0, 1]
[1, 1, 1, 0, 0, 1] (1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0)


ValueError: operands could not be broadcast together with shapes (6,) (12,) 

In [60]:
len(final_states_multiplets)

24

In [61]:
multiplet_d3_down+multiplet_d3_up

[[1, 0, 1, 0, 0, 1],
 [1, 0, 0, 1, 1, 0],
 [0, 1, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1],
 [0, 1, 1, 0, 0, 1],
 [0, 1, 0, 1, 1, 0]]

In [62]:
possible_states_d5

[[1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 0, 1],
 [1, 1, 1, 0, 1, 1],
 [1, 1, 0, 1, 1, 1]]

In [64]:
list(itertools.product(multiplet_d3_down+multiplet_d3_up,possible_states_d5))

[([1, 0, 1, 0, 0, 1], [1, 1, 1, 1, 1, 0]),
 ([1, 0, 1, 0, 0, 1], [1, 1, 1, 1, 0, 1]),
 ([1, 0, 1, 0, 0, 1], [1, 1, 1, 0, 1, 1]),
 ([1, 0, 1, 0, 0, 1], [1, 1, 0, 1, 1, 1]),
 ([1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 0]),
 ([1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 0, 1]),
 ([1, 0, 0, 1, 1, 0], [1, 1, 1, 0, 1, 1]),
 ([1, 0, 0, 1, 1, 0], [1, 1, 0, 1, 1, 1]),
 ([0, 1, 1, 0, 1, 0], [1, 1, 1, 1, 1, 0]),
 ([0, 1, 1, 0, 1, 0], [1, 1, 1, 1, 0, 1]),
 ([0, 1, 1, 0, 1, 0], [1, 1, 1, 0, 1, 1]),
 ([0, 1, 1, 0, 1, 0], [1, 1, 0, 1, 1, 1]),
 ([1, 0, 0, 1, 0, 1], [1, 1, 1, 1, 1, 0]),
 ([1, 0, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1]),
 ([1, 0, 0, 1, 0, 1], [1, 1, 1, 0, 1, 1]),
 ([1, 0, 0, 1, 0, 1], [1, 1, 0, 1, 1, 1]),
 ([0, 1, 1, 0, 0, 1], [1, 1, 1, 1, 1, 0]),
 ([0, 1, 1, 0, 0, 1], [1, 1, 1, 1, 0, 1]),
 ([0, 1, 1, 0, 0, 1], [1, 1, 1, 0, 1, 1]),
 ([0, 1, 1, 0, 0, 1], [1, 1, 0, 1, 1, 1]),
 ([0, 1, 0, 1, 1, 0], [1, 1, 1, 1, 1, 0]),
 ([0, 1, 0, 1, 1, 0], [1, 1, 1, 1, 0, 1]),
 ([0, 1, 0, 1, 1, 0], [1, 1, 1, 0, 1, 1]),
 ([0, 1, 0,